In [15]:
import torch
import os
from dotenv import load_dotenv
import tiktoken
from langchain.embeddings import OpenAIEmbeddings
import pinecone
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [2]:
load_dotenv('./.env')

True

#### Leitura do arquivo e criação do splitter'

In [3]:
with open('docs/nat.txt') as f:
    clt = f.read()

# Criação de um objeto RecursiveCharacterTextSplitter para dividir o texto em pedaços
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10,
    length_function=len
)

In [4]:
chunks = text_splitter.create_documents([clt])

In [5]:
# Função para calcular o valor do embedding usando o ada-002 da OpenAI
def embedding_cost(texts):
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    total_em_dolar = total_tokens / 1000 * 0.0001
    print(f'Total de tokens: {total_tokens}')
    print(f'Custo de Embedding em USD: {total_em_dolar:.6f}')
    print(f'Custo de Embedding em R$: {total_em_dolar * 4.95:.6f}')

In [6]:
embedding_cost(chunks)

Total de tokens: 818
Custo de Embedding em USD: 0.000082
Custo de Embedding em R$: 0.000405


In [7]:
embeddings = OpenAIEmbeddings()

In [8]:
pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

In [9]:
indexes = pinecone.list_indexes()
for i in indexes:
    print('Index encontrada e apagada: ' + i)
    pinecone.delete_index(i)

Index encontrada e apagada: natgpt


In [10]:
index_name = 'natgpt'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536, metric='cosine')
    print('Index '+index_name+' criado')

Index natgpt criado


In [11]:
vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

### Conversando com LLM sobre dados

In [12]:
llm = ChatOpenAI(model='gpt-3.5-turbo-16k', temperature=0.5)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':3})
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [16]:
query = 'me da um resumo sobre o que é o nat'
resp = chain.run(query)

In [17]:
print(resp)

O Núcleo de Apoio Técnico (NAT) é um órgão auxiliar do Ministério Público do Estado do Acre, criado com o objetivo de fornecer suporte técnico, científico e operacional aos membros e órgãos do Ministério Público. Ele foi estabelecido em 2012 e regulado pela Lei Orgânica do Ministério Público do Estado do Acre em 2014. O NAT atende demandas de baixa, média e alta complexidade, tanto em processos judiciais quanto extrajudiciais, e também oferece apoio ao Grupo de Atuação Especial de Combate ao Crime Organizado (GAECO) em suas operações e atividades diárias. Desde a sua criação, o NAT tem sido fundamental para o desenvolvimento das atividades finais da instituição.
